In [55]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


%config InlineBackend.figure_format = 'retina'
import pandas as pd
import networkx as nx
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# Data

In [56]:
X_1=pd.read_csv('data/1_X.csv')
Y_1=np.ravel(pd.read_csv('data/1_Y.csv').replace({-1:0}))

In [124]:
X_1.shape

(3000, 174)

In [142]:
print('Positive rate: ',np.mean(Y_1))

Positive rate:  0.27


In [144]:
null_error=1-max(np.mean(Y_1),1-np.mean(Y_1))
null_error

0.27000000000000002

In [57]:
X_2=pd.read_csv('data/2_X.csv')
Y_2=np.ravel(pd.read_csv('data/2_Y.csv').replace({-1:0}))

In [125]:
X_2.shape

(12145, 4)

In [146]:
print('Positive rate: ',np.mean(Y_2))

Positive rate:  0.110004116921


In [147]:
null_error=1-max(np.mean(Y_2),1-np.mean(Y_2))
null_error

0.11000411692054346

In [133]:
np.mean(Y_3)

0.24385714285714286

In [132]:
X_3=pd.read_csv('data/3_X.csv')
Y_3=np.ravel(pd.read_csv('data/3_Y.csv').replace({-1:0}))

In [126]:
X_3.shape

(7000, 107)

In [148]:
print('Positive rate: ',np.mean(Y_3))

Positive rate:  0.243857142857


In [149]:
null_error=1-max(np.mean(Y_3),1-np.mean(Y_3))
null_error

0.24385714285714288

In [59]:
X_4=pd.read_csv('data/4_X.csv')
Y_4=np.ravel(pd.read_csv('data/4_Y.csv').replace({-1:0}))

In [128]:
X_4.shape

(7000, 115)

In [150]:
print('Positive rate: ',np.mean(Y_4))

Positive rate:  0.531


In [151]:
null_error=1-max(np.mean(Y_4),1-np.mean(Y_4))
null_error

0.46899999999999997

# Function defination 

In [ ]:
def 

In [60]:
Thre=np.arange(0,1,0.05);
C_lists     = [0.1,1,10,100] # Different C to try.
gamma_lists = [0.0000001,0.000001,0.00001,0.0001]# Different gamma to try.
D_list=[1,2,3,4,5,6,7,8,9,10,11,12,13]
test_rate_list=[{0.8:0,0.5:1,0.2:2}]

In [86]:
def myfunc(x,y,test_rate,clf_type,p=False,results=False):
    print(f'Test proportion is {test_rate}\n')
    training_error=[]
    testing_error=[]
    for i in range(0,3):
        if i>=1:
            p = False
        x_train,x_test,y_train,y_test= train_test_split(
            x,y,test_size=test_rate,random_state=0)

        if clf_type=='svm':
            clf=svm_results_(x_train,y_train,p)
            
            training_error.append(error(clf.predict(x_train),y_train,'\nTraining Error ',False))
            testing_error.append(error(clf.predict(x_test),y_test,'\nTraining Error ',False))

        if clf_type=='log':
            clf=log_results_(x_train,y_train,p)
            th=tuning_thre(clf,x_train,y_train,p)
            training_error.append(error(lr_pred(clf,x_train,th),y_train,'\nTraining Error ',False))
            testing_error.append(error(lr_pred(clf,x_test,th),y_test,   'Testing  Error ',False))

        if clf_type=='DT':
            clf=DT_results_(x_train,y_train,p)
            training_error.append(error(clf.predict(x_train),y_train,'\nTraining Error ',False))
            testing_error.append(error(clf.predict(x_test),y_test,'\nTraining Error ',False))
    
    if results == True :
        print('Training Error : ',np.mean(training_error))
        print('Testing Error  : ',np.mean(testing_error))
    return clf,training_error,testing_error

In [74]:
def error (clf_pred,y,error_type,p=True):
    if p==True:
        print(f'{error_type} : ',1-metrics.accuracy_score(y,clf_pred))
    return 1-metrics.accuracy_score(y,clf_pred)

In [75]:
def training_error(clf_list):
    error=0
    for clf in clf_list:
        error+=clf.cv_results_['mean_train_score'].mean()
    return error/len(clf_list)

def testing_error(clf_list):
    error=0
    for clf in clf_list:
        error+=clf.cv_results_['mean_test_score'].mean()
    return error/len(clf_list)
    

### SVM 

In [64]:
def svm_results_(x,y,p=False):
    
    svc=svm.SVC()
    parameters = {'kernel':['rbf'], 'C':C_lists,'gamma':gamma_lists}
    svm_ = GridSearchCV(svc, parameters, cv=3,iid=False,return_train_score =True)
    svm_.fit(x,y);
    if p==True:
        print('Inside cv:')
        print('    Testing  score of  split one :',svm_.cv_results_['split0_test_score'].mean())
        print('    Training score of  split one :',svm_.cv_results_['split0_train_score'].mean())
        print('    Testing  score of  split two :',svm_.cv_results_['split1_test_score'].mean())
        print('    Training score of  split two :',svm_.cv_results_['split1_train_score'].mean())
        print('    Testing  score of  split three :',svm_.cv_results_['split2_test_score'].mean())
        print('    Training score of  split three :',svm_.cv_results_['split2_train_score'].mean())
        print('Mean test score  : ',svm_.cv_results_['mean_test_score'].mean())
        print('Mean train score : ',svm_.cv_results_['mean_train_score'].mean())
        print('Best parameters :',svm_.best_params_)
        
    return svm_

### Logistic Regression

In [65]:
def log_results_(x,y,p=False):
    
    parameters = {'C':C_lists}
    lr = GridSearchCV(LogisticRegression(), parameters, cv=3,iid=False,return_train_score =True)
    lr.fit(x,y);
    if p==True:
        print('Inside cv:')
        print('    Testing  score of  split one :',lr.cv_results_['split0_test_score'].mean())
        print('    Training score of  split one :',lr.cv_results_['split0_train_score'].mean())
        print('    Testing  score of  split two :',lr.cv_results_['split1_test_score'].mean())
        print('    Training score of  split two :',lr.cv_results_['split1_train_score'].mean())
        print('    Testing  score of  split three :',lr.cv_results_['split2_test_score'].mean())
        print('    Training score of  split three :',lr.cv_results_['split2_train_score'].mean())
        print('Mean test score  : ',lr.cv_results_['mean_test_score'].mean())
        print('Mean train score : ',lr.cv_results_['mean_train_score'].mean())
        print('Best parameters :',lr.best_params_)
    return lr

def tuning_thre(lr,x,y,p=False):
    Prob=pd.DataFrame(lr.predict_proba(x)[:,0])
    max_=0
    for th in Thre:
        lr_pred=Prob.applymap(lambda x: 0 if x>th else 1)
        score=metrics.accuracy_score(y,lr_pred)
        if score>max_:
            t=th
            max_=score
    if p==True:
        print('Best threshold : ',t)
    return t

def lr_pred(lr,x,t):
    Prob=pd.DataFrame(lr.predict_proba(x)[:,0])
    lr_pred=Prob.applymap(lambda x: 0 if x>t else 1)
    return lr_pred

### Decision Tree

In [66]:
def DT_results_(x,y,p=False):
    
    parameters=[{'max_depth':D_list}]
    DT_ = GridSearchCV(DecisionTreeClassifier(), parameters,cv=3,iid=False,return_train_score=True)
    DT_.fit(x,y)
    if p==True:
        print('Inside cv:')
        print('    Testing  score of  split one :',DT_.cv_results_['split0_test_score'].mean())
        print('    Training score of  split one :',DT_.cv_results_['split0_train_score'].mean())
        print('    Testing  score of  split two :',DT_.cv_results_['split1_test_score'].mean())
        print('    Training score of  split two :',DT_.cv_results_['split1_train_score'].mean())
        print('    Testing  score of  split three :',DT_.cv_results_['split2_test_score'].mean())
        print('    Training score of  split three :',DT_.cv_results_['split2_train_score'].mean())
        print('Mean test score  : ',DT_.cv_results_['mean_test_score'].mean())
        print('Mean train score : ',DT_.cv_results_['mean_train_score'].mean())
        print('Best parameters :',DT_.best_params_)
        
    return DT_

###  Compare the same classifier through datasets

### Compare through classifiers

# Classification Result

### Dataset 1 

### SVM 

In [79]:
svm_1_t1=myfunc(X_1,Y_1,0.8,'svm',True,results=True)[0]

Test proportion is 0.8

Inside cv:
    Testing  score of  split one : 0.703047263682
    Training score of  split one : 0.90899122807
    Testing  score of  split two : 0.713125
    Training score of  split two : 0.90046875
    Testing  score of  split three : 0.692211055276
    Training score of  split three : 0.911315461347
Mean test score  :  0.702794439653
Mean train score :  0.906925146472
Best parameters : {'C': 0.1, 'gamma': 1e-07, 'kernel': 'rbf'}
Training Error :  0.271666666667
Testing Error  :  0.269583333333


In [81]:
svm_1_t2=myfunc(X_1,Y_1,0.5,'svm',False,results=True)[0];

Test proportion is 0.5

Training Error :  0.00866666666667
Testing Error  :  0.262


In [84]:
svm_1_t3=myfunc(X_1,Y_1,0.2,'svm',False,results=True)[0];

Test proportion is 0.2

Training Error :  0.0275
Testing Error  :  0.246666666667


In [154]:
print('Aver_train_error',(0.271666666667+ 0.00866666666667+0.0275)/3)
print('Aver_test_error',(0.269583333333+0.262+ 0.246666666667)/3)

Aver_train_error 0.10261111111122334
Aver_test_error 0.2594166666666667


### Logistic Regression

In [87]:
lr_1_t1=myfunc(X_1,Y_1,0.8,'log',True,results=True)[0];

Test proportion is 0.8

Inside cv:
    Testing  score of  split one : 0.726368159204
    Training score of  split one : 0.729323308271
    Testing  score of  split two : 0.73
    Training score of  split two : 0.7275
    Testing  score of  split three : 0.72864321608
    Training score of  split three : 0.728179551122
Mean test score  :  0.728337125095
Mean train score :  0.728334286464
Best parameters : {'C': 0.1}
Best threshold :  0.0
Training Error :  0.271666666667
Testing Error  :  0.269583333333


In [88]:
lr_1_t2=myfunc(X_1,Y_1,0.5,'log',False,results=True)[0];

Test proportion is 0.5

Training Error :  0.282
Testing Error  :  0.258


In [89]:
lr_1_t3=myfunc(X_1,Y_1,0.2,'log',False,results=True)[0];

Test proportion is 0.2

Training Error :  0.2725
Testing Error  :  0.26


In [159]:
print('Aver_train_error',( 0.271666666667 + 0.282 + 0.2725 )/3)
print('Aver_test_error',( 0.269583333333 + 0.258 +  0.26)/3)

Aver_train_error 0.275388888889
Aver_test_error 0.26252777777766667


### Decision Tree

In [90]:
DT_1_t1=myfunc(X_1,Y_1,0.8,'DT',True,results=True)[0];

Test proportion is 0.8

Inside cv:
    Testing  score of  split one : 0.851894374282
    Training score of  split one : 0.854443801812
    Testing  score of  split two : 0.819230769231
    Training score of  split two : 0.876730769231
    Testing  score of  split three : 0.814843448009
    Training score of  split three : 0.874928064454
Mean test score  :  0.828656197174
Mean train score :  0.868700878499
Best parameters : {'max_depth': 9}
Training Error :  0.126111111111
Testing Error  :  0.149444444444


In [91]:
DT_1_t2=myfunc(X_1,Y_1,0.5,'DT',False,results=True)[0];

Test proportion is 0.5

Training Error :  0.133333333333
Testing Error  :  0.150666666667


In [92]:
DT_1_t3=myfunc(X_1,Y_1,0.2,'DT',False,results=True)[0];

Test proportion is 0.2

Training Error :  0.12875
Testing Error  :  0.142222222222


In [160]:
print('Aver_train_error',( 0.12875 + 0.133333333333 + 0.126111111111 )/3)
print('Aver_test_error',( 0.149444444444 + 0.150666666667 + 0.142222222222 )/3)

Aver_train_error 0.129398148148
Aver_test_error 0.14744444444433333


### Dataset 2

### SVM-2 

In [93]:
svm_2_t1=myfunc(X_2,Y_2,0.8,'svm',True,results=True)[0];

Test proportion is 0.8

Inside cv:
    Testing  score of  split one : 0.883950617284
    Training score of  split one : 0.897390364422
    Testing  score of  split two : 0.882330246914
    Training score of  split two : 0.897969425571
    Testing  score of  split three : 0.885429542645
    Training score of  split three : 0.896566358025
Mean test score  :  0.883903468948
Mean train score :  0.897308716006
Best parameters : {'C': 100, 'gamma': 1e-06, 'kernel': 'rbf'}
Training Error :  0.0979827089337
Testing Error  :  0.102614244545


In [94]:
svm_2_t2=myfunc(X_2,Y_2,0.5,'svm',False,results=True)[0];

Test proportion is 0.5

Training Error :  0.0981554677207
Testing Error  :  0.100609254075


In [95]:
svm_2_t3=myfunc(X_2,Y_2,0.2,'svm',False,results=True)[0];

Test proportion is 0.2

Training Error :  0.0984973240016
Testing Error  :  0.106628242075


In [155]:
print('Aver_train_error',(0.0979827089337+ 0.0981554677207+0.0984973240016)/3)
print('Aver_test_error',( 0.100609254075+0.102614244545+ 0.106628242075)/3)

Aver_train_error 0.09821183355199999
Aver_test_error 0.103283913565


### Logistic Regression -2

In [96]:
lr_2_t1=myfunc(X_2,Y_2,0.8,'log',True,results=True)[0];

Test proportion is 0.8

Inside cv:
    Testing  score of  split one : 0.884259259259
    Training score of  split one : 0.886195182211
    Testing  score of  split two : 0.885185185185
    Training score of  split two : 0.885731933292
    Testing  score of  split three : 0.886279357231
    Training score of  split three : 0.885648148148
Mean test score  :  0.885241267225
Mean train score :  0.885858421217
Best parameters : {'C': 10}
Best threshold :  0.0
Training Error :  0.114450391107
Testing Error  :  0.108892548374


In [97]:
lr_2_t2=myfunc(X_2,Y_2,0.5,'log',False,results=True)[0];

Test proportion is 0.5

Training Error :  0.0848155467721
Testing Error  :  0.0963280092211


In [98]:
lr_2_t3=myfunc(X_2,Y_2,0.2,'log',False,results=True)[0];

Test proportion is 0.2

Training Error :  0.0936599423631
Testing Error  :  0.0979827089337


In [161]:
print('Aver_train_error',( 0.114450391107 + 0.0848155467721 + 0.0936599423631 )/3)
print('Aver_test_error',(  0.0979827089337+ 0.0963280092211 +0.108892548374  )/3)

Aver_train_error 0.09764196008073334
Aver_test_error 0.10106775550960001


### Decision Tree -2

In [99]:
DT_2_t1=myfunc(X_2,Y_2,0.8,'DT',True,results=True)[0];

Test proportion is 0.8

Inside cv:
    Testing  score of  split one : 0.887464387464
    Training score of  split one : 0.942082006937
    Testing  score of  split two : 0.89648622982
    Training score of  split two : 0.946643227063
    Testing  score of  split three : 0.892554911096
    Training score of  split three : 0.943922127255
Mean test score  :  0.89216850946
Mean train score :  0.944215787085
Best parameters : {'max_depth': 4}
Training Error :  0.08151502676
Testing Error  :  0.0893371757925


In [100]:
DT_2_t2=myfunc(X_2,Y_2,0.5,'DT',False,results=True)[0];

Test proportion is 0.5

Training Error :  0.0858036890646
Testing Error  :  0.0912234480487


In [101]:
DT_2_t3=myfunc(X_2,Y_2,0.2,'DT',False,results=True)[0];

Test proportion is 0.2

Training Error :  0.0865582544257
Testing Error  :  0.0938657883903


In [162]:
print('Aver_train_error',( 0.08151502676 +0.0858036890646  + 0.0865582544257 )/3)
print('Aver_test_error',( 0.0938657883903 + 0.0893371757925 + 0.0912234480487 )/3)

Aver_train_error 0.08462565675009999
Aver_test_error 0.09147547074383333


### Dataset 3

### SVM-3

In [102]:
svm_3_t1=myfunc(X_3,Y_3,0.8,'svm',True,results=True)[0];

Test proportion is 0.8

Inside cv:
    Testing  score of  split one : 0.822537473233
    Training score of  split one : 0.816117363344
    Testing  score of  split two : 0.807146680942
    Training score of  split two : 0.823553054662
    Testing  score of  split three : 0.80364806867
    Training score of  split three : 0.820061563169
Mean test score  :  0.811110740948
Mean train score :  0.819910660392
Best parameters : {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
Training Error :  0.148571428571
Testing Error  :  0.163392857143


In [103]:
svm_3_t2=myfunc(X_3,Y_3,0.5,'svm',False,results=True)[0];

Test proportion is 0.5

Training Error :  0.127428571429
Testing Error  :  0.144571428571


In [104]:
svm_3_t3=myfunc(X_3,Y_3,0.2,'svm',False,results=True)[0];

Test proportion is 0.2

Training Error :  0.130178571429
Testing Error  :  0.139285714286


In [163]:
print('Aver_train_error',( 0.148571428571 + 0.127428571429 + 0.130178571429 )/3)
print('Aver_test_error',( 0.144571428571 + 0.139285714286 +  0.163392857143)/3)

Aver_train_error 0.135392857143
Aver_test_error 0.14908333333333335


### Logistic Regression -3

In [105]:
lr_3_t1=myfunc(X_3,Y_3,0.8,'log',True,results=True)[0];

Test proportion is 0.8

Inside cv:
    Testing  score of  split one : 0.839935760171
    Training score of  split one : 0.862004287245
    Testing  score of  split two : 0.840471092077
    Training score of  split two : 0.855841371919
    Testing  score of  split three : 0.851931330472
    Training score of  split three : 0.855995717345
Mean test score  :  0.844112727573
Mean train score :  0.857947125503
Best parameters : {'C': 10}
Best threshold :  0.4
Training Error :  0.139285714286
Testing Error  :  0.166071428571


In [106]:
lr_3_t2=myfunc(X_3,Y_3,0.5,'log',False,results=True)[0];

Test proportion is 0.5

Training Error :  0.143142857143
Testing Error  :  0.154857142857


In [107]:
lr_3_t3=myfunc(X_3,Y_3,0.2,'log',False,results=True)[0];

Test proportion is 0.2

Training Error :  0.146785714286
Testing Error  :  0.150714285714


In [164]:
print('Aver_train_error',( 0.139285714286 + 0.143142857143 +  0.146785714286)/3)
print('Aver_test_error',( 0.166071428571 + 0.154857142857 + 0.150714285714 )/3)

Aver_train_error 0.1430714285716667
Aver_test_error 0.157214285714


### Decision Tree -3

In [108]:
DT_3_t1=myfunc(X_3,Y_3,0.8,'DT',True,results=True)[0];

Test proportion is 0.8

Inside cv:
    Testing  score of  split one : 0.808598253994
    Training score of  split one : 0.895622062825
    Testing  score of  split two : 0.824575852413
    Training score of  split two : 0.889438535741
    Testing  score of  split three : 0.826180257511
    Training score of  split three : 0.878191401746
Mean test score  :  0.819784787973
Mean train score :  0.88775066677
Best parameters : {'max_depth': 4}
Training Error :  0.142142857143
Testing Error  :  0.158214285714


In [109]:
DT_3_t2=myfunc(X_3,Y_3,0.5,'DT',False,results=True)[0];

Test proportion is 0.5

Training Error :  0.143142857143
Testing Error  :  0.160380952381


In [110]:
DT_3_t3=myfunc(X_3,Y_3,0.2,'DT',False,results=True)[0];

Test proportion is 0.2

Training Error :  0.14
Testing Error  :  0.146428571429


In [165]:
print('Aver_train_error',( 0.143142857143 + 0.142142857143 + 0.14 )/3)
print('Aver_test_error',(  0.158214285714+ 0.160380952381 + 0.146428571429 )/3)

Aver_train_error 0.141761904762
Aver_test_error 0.15500793650800002


### Dataset 4 

### SVM-4

In [111]:
svm_4_t1=myfunc(X_4,Y_4,0.8,'svm',True,results=True)[0];

Test proportion is 0.8

Inside cv:
    Testing  score of  split one : 0.545673076923
    Training score of  split one : 0.551435085837
    Testing  score of  split two : 0.546673819742
    Training score of  split two : 0.547912205567
    Testing  score of  split three : 0.544393776824
    Training score of  split three : 0.549585117773
Mean test score  :  0.545580224497
Mean train score :  0.549644136392
Best parameters : {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
Training Error :  0.376428571429
Testing Error  :  0.431428571429


In [112]:
svm_4_t2=myfunc(X_4,Y_4,0.5,'svm',False,results=True)[0];

Test proportion is 0.5

Training Error :  0.365714285714
Testing Error  :  0.419714285714


In [113]:
svm_4_t3=myfunc(X_4,Y_4,0.2,'svm',False,results=True)[0];

Test proportion is 0.2

Training Error :  0.375714285714
Testing Error  :  0.410714285714


In [157]:
print('Aver_train_error',(0.376428571429+  0.365714285714+0.375714285714)/3)
print('Aver_test_error',(0.431428571429+0.410714285714+ 0.419714285714)/3)

Aver_train_error 0.37261904761899994
Aver_test_error 0.420619047619


### Logistic Regression-4

In [114]:
lr_4_t1=myfunc(X_4,Y_4,0.8,'log',True,results=True)[0];

Test proportion is 0.8

Inside cv:
    Testing  score of  split one : 0.535256410256
    Training score of  split one : 0.679721030043
    Testing  score of  split two : 0.583690987124
    Training score of  split two : 0.658458244111
    Testing  score of  split three : 0.535407725322
    Training score of  split three : 0.674785867238
Mean test score  :  0.551451707568
Mean train score :  0.670988380464
Best parameters : {'C': 0.1}
Best threshold :  0.5
Training Error :  0.351428571429
Testing Error  :  0.4225


In [115]:
lr_4_t2=myfunc(X_4,Y_4,0.5,'log',False,results=True)[0];

Test proportion is 0.5

Training Error :  0.360285714286
Testing Error  :  0.414285714286


In [116]:
lr_4_t3=myfunc(X_4,Y_4,0.2,'log',False,results=True)[0];

Test proportion is 0.2

Training Error :  0.374642857143
Testing Error  :  0.417857142857


In [166]:
print('Aver_train_error',( 0.374642857143 +0.360285714286  + 0.351428571429 )/3)
print('Aver_test_error',(0.4225  + 0.414285714286 + 0.417857142857)/3)

Aver_train_error 0.3621190476193333
Aver_test_error 0.41821428571433333


### Decision Tree-4

In [167]:
DT_4_t1=myfunc(X_4,Y_4,0.8,'DT',True,results=True)[0];

Test proportion is 0.8

Inside cv:
    Testing  score of  split one : 0.519230769231
    Training score of  split one : 0.649554308353
    Testing  score of  split two : 0.539617035325
    Training score of  split two : 0.60904299127
    Testing  score of  split three : 0.523605150215
    Training score of  split three : 0.655493328941
Mean test score  :  0.527484318257
Mean train score :  0.638030209521
Best parameters : {'max_depth': 1}
Training Error :  0.455
Testing Error  :  0.4725


In [118]:
DT_4_t2=myfunc(X_4,Y_4,0.5,'DT',False,results=True)[0];

Test proportion is 0.5

Training Error :  0.464285714286
Testing Error  :  0.473333333333


In [119]:
DT_4_t3=myfunc(X_4,Y_4,0.2,'DT',False,results=True)[0];

Test proportion is 0.2

Training Error :  0.408869047619
Testing Error  :  0.462380952381


In [168]:
print('Aver_train_error',( 0.455 + 0.464285714286 + 0.408869047619 )/3)
print('Aver_test_error',( 0.4725 + 0.473333333333 + 0.462380952381 )/3)

Aver_train_error 0.44271825396833336
Aver_test_error 0.4694047619046667


# Comparison

In [120]:
clf_svm_t1=[svm_1_t1,svm_2_t1,svm_3_t1,svm_4_t1]
clf_svm_t2=[svm_1_t2,svm_2_t2,svm_3_t2,svm_4_t2]
clf_svm_t3=[svm_1_t3,svm_2_t3,svm_3_t3,svm_4_t3]

clf_svm_1=[svm_1_t1,svm_1_t2,svm_1_t3]
clf_svm_2=[svm_2_t1,svm_2_t2,svm_2_t3]
clf_svm_3=[svm_3_t1,svm_3_t2,svm_3_t3]
clf_svm_4=[svm_4_t1,svm_4_t2,svm_4_t3]

clf_lr_t1=[lr_1_t1,lr_2_t1,lr_3_t1,lr_4_t1]
clf_lr_t2=[lr_1_t2,lr_2_t2,lr_3_t2,lr_4_t2]
clf_lr_t3=[lr_1_t3,lr_2_t3,lr_3_t3,lr_4_t3]

clf_lr_1=[lr_1_t1,lr_1_t2,lr_1_t3]
clf_lr_2=[lr_2_t1,lr_2_t2,lr_2_t3]
clf_lr_3=[lr_3_t1,lr_3_t2,lr_3_t3]
clf_lr_4=[lr_4_t1,lr_4_t2,lr_4_t3]

clf_DT_t1=[DT_1_t1,DT_2_t1,DT_3_t1,DT_4_t1]
clf_DT_t2=[DT_1_t2,DT_2_t2,DT_3_t2,DT_4_t2]
clf_DT_t3=[DT_1_t3,DT_2_t3,DT_3_t3,DT_4_t3]

clf_DT_1=[DT_1_t1,DT_1_t2,DT_1_t3]
clf_DT_2=[DT_2_t1,DT_2_t2,DT_2_t3]
clf_DT_3=[DT_3_t1,DT_3_t2,DT_3_t3]
clf_DT_4=[DT_4_t1,DT_4_t2,DT_4_t3]